# <img src="../images/sberbank.png">
# <center> Финальное задание </center>
## <center> Предсказание пола клиента по транзакциям</center>

## Описание задачи 
### В рамках финального задания будет необходимо предсказывать пол клиента, основываясь на его транзакционных исторических данных. Выполнение финального задания - это маленький шаг в большую Data Science-всесенную, поэтому отнеситесь к нему максимально серьёзно :)
### Вы будете строить предиктивные модели и отправлять результаты своего моделирования на платформу [Kaggle](https://www.kaggle.com/t/e8a939488d274dab9051cce14d5ca952), где и будет оцениваться каждое решение и положение участников. Но переживать не стоит - код, связанный с построением модели мы уже написали, поэтому вашим основным заданием будет создание новых переменных для генерации новых инсайдов из данных, которые смогут улучшить полученные значения метрики.
### В роли метрики выступает [ROC AUC](https://dyakonov.org/2017/07/28/auc-roc-%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C-%D0%BF%D0%BE%D0%B4-%D0%BA%D1%80%D0%B8%D0%B2%D0%BE%D0%B9-%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA/), который и нужно будет оптимизировать.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re
import matplotlib.pyplot as plt

from tqdm._tqdm_notebook import tqdm_notebook
from warnings import filterwarnings

%matplotlib inline
filterwarnings('ignore')

In [ ]:
# Считываем данные
tr_mcc_codes = pd.read_csv('../data/tr_mcc_codes.csv', sep=';', index_col='mcc_code')
tr_types = pd.read_csv('../data/tr_types.csv', sep=';', index_col='tr_type')

transactions = pd.read_csv('../data/transactions.csv', index_col='customer_id')
gender_train = pd.read_csv('../data/gender_train.csv', index_col='customer_id')
gender_test = pd.read_csv('../data/gender_test.csv', index_col='customer_id')
transactions_train = transactions.join(gender_train, how='inner')
transactions_test = transactions.join(gender_test, how='inner')

del transactions

In [ ]:
#transactions_test.info()

In [ ]:
#transactions_train.head()

In [376]:
# Функции, которыми можно пользоваться для построения классификатора, 
# оценки его результатов и построение прогноза для тестовой части пользователей

# Cross-validation score (среднее значение метрики ROC AUC на тренировочных данных)
def cv_score(params, train, y_true, num_boost):
    cv_res=xgb.cv(params, xgb.DMatrix(train, y_true),
                  early_stopping_rounds=10, maximize=True, 
                  num_boost_round=num_boost, nfold=5, stratified=True)
#                  num_boost_round=10000, nfold=5, stratified=True)
#    print(cv_res)
    index_argmax = cv_res['test-auc-mean'].argmax()
    print('Cross-validation, ROC AUC: {:.3f}+-{:.3f}, Trees: {}'.format(cv_res.loc[index_argmax]['test-auc-mean'],
                                                                        cv_res.loc[index_argmax]['test-auc-std'],
                                                                        index_argmax))
    return cv_res.loc[index_argmax]['test-auc-mean'], cv_res.loc[index_argmax]['test-auc-std']

# Построение модели + возврат результатов классификации тестовых пользователей
def fit_predict(params, num_trees, train, test, target):
    params['learning_rate'] = params['eta']
    clf = xgb.train(params, xgb.DMatrix(train.values, target, feature_names=list(train.columns)), 
                    num_boost_round=num_trees, maximize=True)
    y_pred = clf.predict(xgb.DMatrix(test.values, feature_names=list(train.columns)))
    submission = pd.DataFrame(index=test.index, data=y_pred, columns=['probability'])
    return clf, submission

# Отрисовка важности переменных. Важность переменной - количество разбиений выборки, 
# в которых участвует данная переменная. Чем больше - тем она, вероятно, лучше 
def draw_feature_importances(clf, top_k=10):
    plt.figure(figsize=(10, 10))
    
    importances = dict(sorted(clf.get_score().items(), key=lambda x: x[1])[-top_k:])
    y_pos = np.arange(len(importances))
    
    plt.barh(y_pos, list(importances.values()), align='center', color='green')
    plt.yticks(y_pos, importances.keys(), fontsize=12)
    plt.xticks(fontsize=12)
    plt.xlabel('Feature importance', fontsize=15)
    plt.title('Features importances, Sberbank Gender Prediction', fontsize=18)
    plt.ylim(-0.5, len(importances) - 0.5)
    plt.show()

### Так как код для оценки модели на тренировочных данных и её применения на тестовых данных уже дан, то мы будем работать над тем, чтобы создать переменные для улучшения результатов моделирования. 

### (!) В рамках данного задания Вы можете делать всё, что угодно - использовать другие алгоритмы и/или их комбинации, подбирать гиперпараметры своих моделей, отбирать переменые, etc. Мы создали шаблон для простоты и для Вашего понимания верхнеуровневого процесса разработки модели, опустив при этом большое число деталей.

## Basic features
Начнём с того, что сформируем базовые переменные по каждому пользователю. На этом этапе будем использовать стандартные агрегации, посчитанные на расходах и приходах клиента:
- минимум
- максимум
- среднее
- медиана
- среднеквадратичное отклонение
- количество

Также параметры модели выберем стандартные, запишем их в словарь params, и будем использовать для дальнейшего построения модели (не забывайте, что с этим Вы можете тоже экспериментировать).

In [ ]:
params = {
    'eta': 0.1,
    'max_depth': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    
    'gamma': 0,
    'lambda': 0,
    'alpha': 0,
    'min_child_weight': 0,
    
    'eval_metric': 'auc',
    'objective': 'binary:logistic' ,
    'booster': 'gbtree',
    'njobs': -1,
    'tree_method': 'approx'
}

In [ ]:
tqdm_notebook.pandas(desc="Progress:")

In [ ]:
def features_creation_basic(x): 
    features = []
    features.append(pd.Series(x[x['amount']>0]['amount'].agg(['min', 'max', 'mean', 'median', 'std', 'count'])\
                                                        .add_prefix('positive_transactions_')))
    features.append(pd.Series(x[x['amount']<0]['amount'].agg(['min', 'max', 'mean', 'median', 'std', 'count'])\
                                                        .add_prefix('negative_transactions_')))
 
    return pd.concat(features)

data_train = transactions_train.groupby(transactions_train.index).progress_apply(features_creation_basic)
data_test = transactions_test.groupby(transactions_test.index).progress_apply(features_creation_basic)

In [ ]:
#data_test.head()

In [ ]:
#data_train.head()

In [ ]:
target = data_train.join(gender_train, how='inner')['gender']
cv_score(params, data_train, target)

In [ ]:
#target.head()

In [ ]:
### Число деревьев для XGBoost имеет смысл выставлять по результатам на кросс-валидации 
clf, submission = fit_predict(params, 88, data_train, data_test, target)
#clf, submission = fit_predict(params, 70, data_train, data_test, target)

In [ ]:
draw_feature_importances(clf, 10)

In [ ]:
submission.to_csv('../data/basic_features_submission.csv')

### Видим, что результат на кросс-валидации - 62.5% ROC AUC.

## Advanced features
Добавим дополнительные переменные по каждому пользователю в модель. <br>
Для этого будем анализировать дни недели, часы и состояние дня/ночи во время покупки - в каждом из случаев будем считать частоту транзакций в соответствующей категории относитеьно всех остальных категорий. <br>
То есть если, например, клиент в 70% случае совершал ночные траты, то мы получим вектор [0.7, 0.3] для этого случая в качестве частот транзакций ночью/днём.

In [ ]:
for df in [transactions_train, transactions_test]:
    df['day'] = df['tr_datetime'].str.split().apply(lambda x: int(x[0]) % 7)
    df['hour'] = df['tr_datetime'].apply(lambda x: re.search(' \d*', x).group(0)).astype(int)
    df['night'] = ~df['hour'].between(6, 22).astype(int)
    df['mcc_tr'] = df['mcc_code'].astype(str)+df['tr_type'].astype(str)
    df['mcc_hour'] = df['mcc_code'].astype(str)+df['tr_datetime'].apply(lambda x: re.search(' \d*', x).group(0)).astype(str)

In [ ]:
#transactions_train.head()

In [ ]:
#transactions_train.info()

In [ ]:
def features_creation_advanced(x): 
    features = []
    features.append(pd.Series(x['day'].value_counts(normalize=True).add_prefix('day_')))
    features.append(pd.Series(x['hour'].value_counts(normalize=True).add_prefix('hour_')))
    features.append(pd.Series(x['night'].value_counts(normalize=True).add_prefix('night_')))
    features.append(pd.Series(x[x['amount']>0]['amount'].agg(['min', 'max', 'mean', 'median', 'std', 'count'])\
                                                        .add_prefix('positive_transactions_')))
    features.append(pd.Series(x[x['amount']<0]['amount'].agg(['min', 'max', 'mean', 'median', 'std', 'count'])\
                                                        .add_prefix('negative_transactions_')))
 
    return pd.concat(features)

In [ ]:
data_train = transactions_train.groupby(transactions_train.index)\
                               .progress_apply(features_creation_advanced).unstack(-1)
data_test = transactions_test.groupby(transactions_test.index)\
                             .progress_apply(features_creation_advanced).unstack(-1)

In [ ]:
#data_train.head()

In [ ]:
target = data_train.join(gender_train, how='inner')['gender']
cv_score(params, data_train, target)

In [ ]:
### Число деревьев для XGBoost имеет смысл выятавлять по результатам на кросс-валидации 
clf, submission = fit_predict(params, 148, data_train, data_test, target)
#clf, submission = fit_predict(params, 70, data_train, data_test, target)

In [ ]:
draw_feature_importances(clf, 10)

### Добавление новых переменных улучшило наши результаты ROC AUC с 62.5% до 68.2%, на тестовой выборке результат будет аналогичным, так что мы явно не переобучились. При этом есть куда стремиться!

### В итоге можем отправить полученное решение на платформу в Kaggle In-Class Competition. Для этого выгрузим его в *.csv - файл, после чего полученный файл можем загружать в качестве ответа.

In [ ]:
submission.to_csv('../data/submission_advanced.csv')

In [ ]:
#transactions_train.index

# (!) Цель задания:
## Полученная модель должна иметь ROC AUC на Public-части тестовой выборки (на лидерборде) не менее 80%.

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import gc

In [ ]:
params = {
    'eta': 0.1,
    'tree_method': "approx", #"hist"
    'grow_policy': "lossguide",
    'max_leaves': 1000,  
    'max_depth': 3, 
    'subsample': 0.9, 
    'alpha':1,
    'objective': 'binary:logistic', 
    'scale_pos_weight':100,
    'eval_metric': 'auc', 
    'nthread':4,
    'silent': 0
}

In [380]:
#dmtrain= xgb.DMatrix(data_train.values, target, feature_names=list(data_train.columns))
#dmvalid= xgb.DMatrix(data_test.values, feature_names=list(data_train.columns))
dmtrain= xgb.DMatrix(data_train[:train_size].values, target[:train_size], feature_names=list(data_train.columns))
dmvalid= xgb.DMatrix(data_train[train_size:].values, target[train_size:], feature_names=list(data_train.columns))

In [381]:
def objective(params):
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    watchlist = [(dmtrain, 'train'), (dmvalid, 'valid')]
    model = xgb.train(params, dmtrain, num_round, watchlist, maximize=True, early_stopping_rounds=20, verbose_eval=0)
    pred = model.predict(dmvalid, ntree_limit=model.best_ntree_limit)
    auc = roc_auc_score(dmvalid.get_label(), pred)
    del model, pred
    gc.collect()
    print(f"SCORE: {auc}")
#    print("params: ",params)
    return { 'loss': 1-auc, 'status': STATUS_OK }

In [382]:
# hyperparameter optimization space
# find more parameters in docs https://github.com/dmlc/xgboost/blob/443ff746e9723dcf571769b0d6ea28fbcb3e4a3f/doc/parameter.md
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 10000, 50),
    #'n_estimators': 3, # WARNING: increse number of estimators, e.g. uncomment the above line (it's small for the sake of example)
    'eta': hp.quniform('eta', 0.025, 1, 0.025),
    'max_depth': hp.choice('max_depth', np.arange(1, 14, dtype=int)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),  
    'subsample': hp.quniform('subsample', 0, 1, 0.05),
    'gamma': hp.quniform('gamma', 0, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0, 1, 0.05),
    'alpha' : hp.quniform('alpha', 0, 10, 1),
    'lambda': hp.quniform('lambda', 1, 2, 0.1),
#    'scale_pos_weight': hp.quniform('scale_pos_weight', 0, 200, 10),
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': "approx", #"hist",
    'sketch_eps': hp.quniform('sketch_eps', 0.03, 1, 0.05),
    'booster': 'gbtree',
#    'nthread': 4, 
    'silent': 0
}

In [383]:
trials = Trials()
max_auc = 0

In [ ]:
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10000, # WARNING: increase number of evaluations (it's small for the sake of example)
    trials=trials
)
print("\n\n\n The best hyperparameters:")
print(best)

SCORE: 0.7990721146953403                                                      
SCORE: 0.873983770609319                                                       
SCORE: 0.8269404444444446                                                      
SCORE: 0.8747437419354838                                                      
SCORE: 0.5                                                                     
SCORE: 0.8526263512544802                                                      
SCORE: 0.8443995985663082                                                      
SCORE: 0.8689913118279571                                                      
SCORE: 0.8489356845878137                                                      
SCORE: 0.837567311827957                                                       
SCORE: 0.8724994982078853                                                      
SCORE: 0.794717247311828                                                       
SCORE: 0.864524845878136                

SCORE: 0.8388615340501793                                                      
SCORE: 0.867837017921147                                                       
SCORE: 0.8702045591397849                                                      
SCORE: 0.8690075985663082                                                      
SCORE: 0.8582168315412186                                                      
SCORE: 0.8360711111111111                                                      
SCORE: 0.8706429247311829                                                      
SCORE: 0.8735874982078852                                                      
SCORE: 0.8688318853046594                                                      
SCORE: 0.8745900501792114                                                      
SCORE: 0.8382274982078852                                                      
SCORE: 0.8615095053763442                                                      
SCORE: 0.8386617347670251               

SCORE: 0.854909247311828                                                       
SCORE: 0.8723601433691756                                                      
SCORE: 0.8767077849462366                                                      
SCORE: 0.863172358422939                                                       
SCORE: 0.8741406738351255                                                      
SCORE: 0.8768029820788531                                                      
SCORE: 0.8598172903225807                                                      
SCORE: 0.8726491756272402                                                      
SCORE: 0.8613913691756273                                                      
SCORE: 0.873500329749104                                                       
SCORE: 0.8681728458781364                                                      
SCORE: 0.8754262939068099                                                      
SCORE: 0.8670499784946235               

SCORE: 0.8556278136200717                                                      
SCORE: 0.8592566594982078                                                      
SCORE: 0.8635042867383513                                                      
SCORE: 0.8752033261648744                                                      
SCORE: 0.8617253620071684                                                      
SCORE: 0.8715792974910395                                                      
SCORE: 0.876935569892473                                                       
SCORE: 0.8423499928315413                                                      
SCORE: 0.864517734767025                                                       
SCORE: 0.872708817204301                                                       
SCORE: 0.8445020215053762                                                      
SCORE: 0.8769676845878136                                                      
SCORE: 0.8740856200716847               

SCORE: 0.8778730896057348                                                      
SCORE: 0.8675257347670251                                                      
SCORE: 0.8732716272401433                                                      
SCORE: 0.8759475842293907                                                      
SCORE: 0.8686302508960574                                                      
SCORE: 0.8723041720430107                                                      
SCORE: 0.8753102222222222                                                      
SCORE: 0.8773615483870967                                                      
SCORE: 0.8771819354838711                                                      
SCORE: 0.873536688172043                                                       
SCORE: 0.8489164157706093                                                      
SCORE: 0.8754043870967743                                                      
SCORE: 0.8690201003584229               

...

SCORE: 0.8797561577060932                                                         
SCORE: 0.8768586093189963                                                         
SCORE: 0.8809879856630825                                                         
SCORE: 0.8779529175627241                                                         
SCORE: 0.8816956559139784                                                         
SCORE: 0.8798474551971326                                                         
SCORE: 0.8348898637992831                                                         
SCORE: 0.8677699211469534                                                         
SCORE: 0.8768578064516128                                                         
SCORE: 0.8781389534050179                                                         
SCORE: 0.863541906810036                                                          
SCORE: 0.8756193261648746                                                         
SCOR

SCORE: 0.8761196272401434                                                         
SCORE: 0.8714135627240144                                                         
SCORE: 0.8816956559139784                                                         
SCORE: 0.866878394265233                                                          
SCORE: 0.8772775913978493                                                         
SCORE: 0.875079340501792                                                          
SCORE: 0.8743505663082437                                                         
SCORE: 0.8776636559139783                                                         
SCORE: 0.8822035268817203                                                         
SCORE: 0.8782862222222223                                                         
SCORE: 0.8781754265232974                                                         
SCORE: 0.8817119426523297                                                         
SCOR

SCORE: 0.5                                                                        
SCORE: 0.8786784802867385                                                         
SCORE: 0.8822035268817203                                                         
SCORE: 0.8783538924731182                                                         
SCORE: 0.8751135197132617                                                         
SCORE: 0.8723274551971325                                                         
SCORE: 0.8782300215053763                                                         
SCORE: 0.8781341362007168                                                         
SCORE: 0.8793157275985664                                                         
SCORE: 0.8822035268817203                                                         
SCORE: 0.8778751541218638                                                         
SCORE: 0.8781554695340503                                                         
SCOR

SCORE: 0.8781676272401434                                                         
SCORE: 0.879937605734767                                                          
SCORE: 0.8765855197132617                                                         
SCORE: 0.8803225232974909                                                         
SCORE: 0.8698102365591397                                                         
SCORE: 0.8742952831541219                                                         
SCORE: 0.867842064516129                                                          
SCORE: 0.8822035268817203                                                         
SCORE: 0.8778237706093188                                                         
SCORE: 0.879937605734767                                                          
SCORE: 0.8713343082437275                                                         
SCORE: 0.8745258207885305                                                         
SCOR

SCORE: 0.8732451326164875                                                         
SCORE: 0.8736105519713262                                                         
SCORE: 0.8798192401433692                                                         
SCORE: 0.868925476702509                                                          
SCORE: 0.878019211469534                                                          
SCORE: 0.879869935483871                                                          
SCORE: 0.8757409032258066                                                         
SCORE: 0.8781001863799283                                                         
SCORE: 0.8709535197132616                                                         
SCORE: 0.8780341218637993                                                         
SCORE: 0.8764279283154123                                                         
SCORE: 0.8786711397849462                                                         
SCOR

In [ ]:
#params = best
best

In [373]:
params = {
    'eta': 0.1, #0.1, #0.1
    'max_depth': 1, #13, #3
    'subsample': 0.85, #0.8
    'colsample_bytree': 0.15, #0.7,
    'n_estimators': 600.0,
    'gamma': 0.45, #0.5,
    'lambda': 1.4, #1.4,
    'alpha': 1, #0,
    'min_child_weight': 2, #4, #0
    
    'eval_metric': 'auc',
    'objective': 'binary:logistic' ,
    'booster': 'gbtree',
    'njobs': -1,
    'tree_method': 'approx',
    'nthread': 4,
    'scale_pos_weight': 60, #170.0
    'sketch_eps': 0.2
}

In [ ]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(3,12)
    for min_child_weight in range(0,8)
]

In [ ]:
num_boost_round = 999
len (gridsearch_params)

In [ ]:
max_auc = 0
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
#    cv_results = xgb.cv(
#        params,
#        dmtrain,
#        num_boost_round=num_boost_round,
#        seed=42,
#        nfold=5,
#        metrics={'auc'},
#        early_stopping_rounds=73
        #73
 #   )
    cv_results = xgb.cv(params, xgb.DMatrix(data_train, target),
                  early_stopping_rounds=100, maximize=True, 
                  num_boost_round=10000, nfold=5, stratified=True)
    
    # Update best MAE
#    mean_mae = cv_results['test-mae-mean'].min()    
    mean_auc = cv_results['test-auc-mean'].max()
#    boost_rounds = cv_results['test-mae-mean'].argmin()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
#    if mean_mae < min_mae:    
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, AUC: {}".format(best_params[0], best_params[1], max_auc))

In [ ]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [ ]:
max_auc = 0
best_params = None
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
#    cv_results = xgb.cv(
#        params,
#        dmtrain,
#        num_boost_round=num_boost_round,
#        seed=42,
#        nfold=5,
#        metrics={'auc'},
#        early_stopping_rounds=73
#    )
    cv_results = xgb.cv(params, xgb.DMatrix(data_train, target),
                  early_stopping_rounds=100, maximize=True, 
                  num_boost_round=10000, nfold=5, stratified=True)
    
    mean_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = (subsample,colsample)
print("Best params: {}, {}, AUC: {}".format(best_params[0], best_params[1], max_auc))

In [365]:
def features_creation_my1(x): 
    features = []
#Cross-validation, ROC AUC: 0.876+-0.010, Trees: 309
    features.append(pd.Series(x['mcc_code'].value_counts(normalize=True).add_prefix('mcc_')))

#Cross-validation, ROC AUC: 0.853+-0.010, Trees: 647
#    features.append(pd.Series(x['mcc_hour'].value_counts(normalize=True).add_prefix('mcch_')))

#Cross-validation, ROC AUC: 0.705+-0.019, Trees: 238
    features.append(pd.Series(x['tr_type'].value_counts(normalize=True).add_prefix('tr_')))
#    features.append(pd.Series(x['mcc_tr'].value_counts(normalize=True).add_prefix('mcc_tr_')))
    features.append(pd.Series(x['day'].value_counts(normalize=True).add_prefix('day_')))
    features.append(pd.Series(x['hour'].value_counts(normalize=True).add_prefix('hour_')))
    features.append(pd.Series(x['night'].value_counts(normalize=True).add_prefix('night_')))
    features.append(pd.Series(x[x['amount']>0]['amount'].agg(['min', 'max', 'mean', 'median', 'std', 'count'])\
                                                        .add_prefix('positive_transactions_')))
    features.append(pd.Series(x[x['amount']<0]['amount'].agg(['min', 'max', 'mean', 'median', 'std', 'count'])\
                                                        .add_prefix('negative_transactions_')))
 
    return pd.concat(features)

In [366]:
data_train = transactions_train.groupby(transactions_train.index)\
                               .progress_apply(features_creation_my1).unstack(-1)
data_test = transactions_test.groupby(transactions_test.index)\
                             .progress_apply(features_creation_my1).unstack(-1)

In [ ]:
def opt_params(param_range, text_opt):
    max_auc = 0
    best_params = None
    for param_x in param_range:
        print("CV with ", text_opt,"={}".format(param_x))
        # We update our parameters
        params[text_opt] = param_x
        # Run CV
        cv_results = xgb.cv(
            params,
            dmtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics={'auc'},
            early_stopping_rounds=73
            )
        # Update best MAE
        mean_auc = cv_results['test-auc-mean'].max()
        boost_rounds = cv_results['test-auc-mean'].argmax()
        print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
        if mean_auc > max_auc:
            max_auc = mean_auc
            best_params = param_x
    return text_opt, best_params, max_auc

In [ ]:
def opt_params_cv(param_range, text_opt):
    max_auc = 0
    best_params = None
    for param_x in param_range:
        print("CV with ", text_opt,"={}".format(param_x))
        # We update our parameters
        params[text_opt] = param_x
        # Run CV
        mean_auc = cv_score(params, data_train, target)[0]
        if mean_auc > max_auc:
            max_auc = mean_auc
            best_params = param_x
    return text_opt, best_params, max_auc

In [ ]:
num_boost_round = 999
opt_params_cv([.3, .2, .1, .05, .01, .005], "eta")
#print("Best params: {}, AUC: {}".format(best_params, max_auc))

In [ ]:
print ("best param -",opt_params_cv(np.arange(0.5, 1, 0.05), "gamma")) #'gamma', 0.5, 1, 0.05

In [ ]:
print ("best param -",opt_params(range(0, 10, 1), "alpha")) #'alpha', 0, 10, 1

In [ ]:
#print ("best param -",opt_params(np.arange(0, 2, 0.1), "lambda")) #'lambda', 1, 2, 0.1
print ("best param -",opt_params_cv(np.arange(0, 2, 0.1), "lambda")) #'lambda', 1, 2, 0.1

In [ ]:
#print ("best param -",opt_params(range(0, 200, 10), "scale_pos_weight")) #'scale_pos_weight', 50, 200, 10

In [341]:
params = {
    'eta': 0.1, #0.1
    'max_depth': 3, #3
    'subsample': 0.7, #0.8
    'colsample_bytree': 0.8,
    
    'gamma': 0.55, #0
    'lambda': 0,
    'alpha': 0,
    'min_child_weight': 4, #0
    
    'eval_metric': 'auc',
    'objective': 'binary:logistic' ,
    'booster': 'gbtree',
    'njobs': -1,
    'tree_method': 'approx'
}

In [374]:
target = data_train.join(gender_train, how='inner')['gender']
#cv_score(xgb_reg_params, data_train, target)

In [377]:
cv_score(params, data_train, target, 700) #NUM_boost !!!!

Cross-validation, ROC AUC: 0.868+-0.008, Trees: 520


(0.8675158, 0.00816313928338848)

In [ ]:
### Число деревьев для XGBoost имеет смысл выятавлять по результатам на кросс-валидации 
clf, submission = fit_predict(params, 372, data_train, data_test, target)
#clf, submission = fit_predict(params, 70, data_train, data_test, target)

In [ ]:
#draw_feature_importances(clf, 10)

In [ ]:
submission.to_csv('../data/gender_test_kaggle_sample_submission.csv')